# common

In [ ]:
// // test

open testing

()



## pair

In [ ]:
inl pair x y =
    x, y

()



In [ ]:
// // test

pair 1i32 2i32
|> _equal (1, 2)

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{struct (1, 2)} / expected: %A{struct (1, 2)}"
    ()
method0()



## ||>

In [ ]:
inl (||>) (arg1, arg2) fn = arg2 |> fn arg1

()



In [ ]:
// // test

(3i32, 2i32)
||> fun a b => a - b
|> _equal 1

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{1} / expected: %A{1}"
    ()
method0()



## flip

In [ ]:
inl flip fn a b =
    fn b a

()



In [ ]:
// // test

(1i32, 2i32)
||> flip pair
|> _equal (2, 1)

let rec method0 () : unit =
    let v0 : string = $"_equal / actual: %A{struct (2, 1)} / expected: %A{struct (2, 1)}"
    ()
method0()

